In [ ]:
from openai import OpenAI
client = OpenAI()

In [13]:
from typing_extensions import override
from openai import AssistantEventHandler
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
 


In [ ]:
assistant = client.beta.assistants.create(
  name="Math Tutor",
  instructions="You are a personal math tutor. Write and run code to answer math questions. Always run the code to answer.",
  tools=[{"type": "code_interpreter"}],
  model="gpt-4-turbo",
)

In [8]:
thread = client.beta.threads.create()

In [10]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="Plot a scatterplot with these data: x=[1,2,3,4,5], y=[10,11,12,13,14]"
)

In [7]:
message

Message(id='msg_2GGuLMsE6oF3a5236TNc3UEd', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1714150039, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_fVgN53S8LrxsDxvuCx1yox1J')

In [ ]:
pd.DataFrame({'CPI':

In [11]:
# message = client.beta.threads.messages.create(
#   thread_id=thread.id,
#   role="user",
#   content="Create a pandas dataframe with the following columns. DATE: month end dates from January 2023 until January 2024. CPI: random values between -1 and 1. SPX: values from 100 to 200 in crescent order but add some randomness  "
# )

In [12]:
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > code_interpreter

import matplotlib.pyplot as plt

# Given data
x = [1, 2, 3, 4, 5]
y = [10, 11, 12, 13, 14]

# Create a scatter plot
plt.figure(figsize=(8, 5))
plt.scatter(x, y, color='blue', marker='o')
plt.title('Scatter Plot')
plt.xlabel('x')
plt.ylabel('y')
plt.grid(True)
plt.show()

output >

assistant > Here is the scatter plot for the data you provided, with \( x = [1, 2, 3, 4, 5] \) and \( y = [10, 11, 12, 13, 14] \). If you need any modifications or additional analysis, please let me know!

In [19]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="Create a dataframe with 3 columns: 'DATE', 'CPI' and 'SPX'. 10 consecutive end of month dates,\
  and plausible values for CPI and SPX. Save the results in a global variable I can access in this script."
)

with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > code_interpreter

# Creating a DataFrame as specified and saving it as a global variable
def create_data():
    global df  # Define df as a global variable
    
    # Creating a date range for 10 consecutive end of month dates
    dates = pd.date_range(start="2023-01-31", periods=10, freq='M')

    # Generating plausible values for CPI and SPX
    cpi_values = np.random.normal(loc=260, scale=3, size=10)  # Simulating CPI centered around 260
    spx_values = np.random.normal(loc=4200, scale=150, size=10)  # Simulating S&P 500 index centered around 4200

    # Creating the DataFrame
    data = {
        'DATE': dates,
        'CPI': cpi_values,
        'SPX': spx_values
    }
    df = pd.DataFrame(data)

create_data()  # Initialize the DataFrame globally
df  # Display the DataFrame

output >

        DATE         CPI          SPX
0 2023-01-31  258.131433  4382.189997
1 2023-02-28  257.197114  4462.182440
2 2023-03-31  255.573507  4308.738685
3 2023-04-30  261.715131  4112.92

In [26]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content=" Fotgetting how this dataframe was created. Is there any correlation between CPI levels and SPX levels one month ahead? Give me a statistical analysis plotting results and statistical summary tables "
)

with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > To analyze the correlation between CPI levels and SPX levels one month ahead, I will shift the SPX column by one month so we can calculate the correlation between CPI at a given month and SPX in the following month. Here are the steps I'll perform:

1. Shift the SPX data by one month.
2. Calculate the correlation between the shifted SPX values and the current CPI values.
3. Provide statistical summaries for both shifted and original data.
4. Plot the data points to visually inspect the correlation.

Let's carry out this analysis.
assistant > code_interpreter

# Create a copy of the dataframe for manipulation
df_shifted = df.copy()

# Shift the SPX data by one month (moving it one row up)
df_shifted['SPX_shifted'] = df_shifted['SPX'].shift(-1)

# Calculate correlation between current CPI and SPX one month ahead
correlation = df_shifted['CPI'].corr(df_shifted['SPX_shifted'])

# Statistical summaries
stat_summary_cpi = df_shifted['CPI'].describe()
stat_summary_spx_shifted = d

In [27]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content=" Can you display the plot results? if you're unable or unsure, can you save a png file in this workign directory? "
)

with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > code_interpreter

# Replotting the data for visual display or saving it as a PNG file if needed

fig, ax = plt.subplots(figsize=(12, 6))
ax.scatter(df_shifted['DATE'], df_shifted['CPI'], color='blue', label='CPI')
ax.scatter(df_shifted['DATE'], df_shifted['SPX_shifted'], color='red', label='SPX (next month)', alpha=0.6)
ax.set_title('CPI and Shifted SPX Over Time')
ax.set_xlabel('Date')
ax.set_ylabel('Index Value')
ax.legend()
plt.grid(True)

# Save the plot as a PNG file in the current working directory
plot_path = '/mnt/data/CPI_and_SPX_Correlation.png'
plt.savefig(plot_path)
plt.close()

plot_path

output >

'/mnt/data/CPI_and_SPX_Correlation.png'

assistant > I have saved the plot as a PNG file which displays the CPI and the SPX levels (shifted one month ahead). You can download the file from the following link:

[Download CPI and SPX Correlation Plot](sandbox:/mnt/data/CPI_and_SPX_Correlation.png)

Please let me know if there is anything else you need!None

## Without streaming

In [9]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)

In [7]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_TgJeFa99RhTwmYBmi50Rl1nD', assistant_id='asst_MnZkDAPQFYW72gyJivdPnmHU', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='After solving the equation \\(3x + 11 = 14\\), we find that \\(x = 1\\). If you have any more questions or need further assistance, feel free to ask, Jane Doe!'), type='text')], created_at=1713631724, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_a4FIJBV68wF6cceZ3r4081Ze', status=None, thread_id='thread_i7ZeJHttx790f62MpEYisdCN'), Message(id='msg_O2KXKTN9suoEnYdVbsUknoY5', assistant_id='asst_MnZkDAPQFYW72gyJivdPnmHU', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Certainly, Jane Doe! To solve the equation \\(3x + 11 = 14\\), let's first isolate the variable \\(x\\). We'll start by subtracting 11 from both sides of the equation. Let's do that now."), type='text')], created_at=1713631